In [ ]:
from google.colab import drive
drive.mount('/gdrive')
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import Input, layers, models, datasets, utils, losses, optimizers 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential

from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D
#from tensorflow.keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import LeakyReLU

In [ ]:
import os
from PIL import Image  # or you can use the keras one to load images
def load_dataset(top_dir = "input_data"):
    images_dataset = []
    name_sorted = []
    for root, dirs, files in os.walk(top_dir):
        for name in files:
            name_sorted.append(name)
        name_sorted = np.sort(name_sorted)
        print(name_sorted) 

            #print(os.path.join(root, name))
        for n in name_sorted:
          img = np.array(Image.open(os.path.join(root, n)))
          images_dataset.append(img)
            
    
    return images_dataset

In [ ]:
data = load_dataset(top_dir = '/content/drive/MyDrive/imagesdata/MRIIMAGES')
X = np.array(data)


In [ ]:
plt.imshow(X[12], cmap = 'gray')


In [ ]:
age = pd.read_csv('/content/drive/MyDrive/imagesdata/participants.csv')

In [ ]:

X = X.reshape(-1, 189, 157, 1)


In [ ]:
X = X/255.

In [ ]:
X  = X.astype('float32')
y = np.array(age['age']).astype('float32')



In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, random_state = 75)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), strides = 1, activation='linear',input_shape=(189,157,1),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2, 2), strides = 2, padding='same'))

model.add(Conv2D(64, (2, 2), strides =1, activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),strides = 2, padding='same'))

model.add(Conv2D(128, (2, 2), activation='linear', strides = 1, padding='same'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(MaxPooling2D(pool_size=(2, 2), strides = 2, padding='same'))

model.add(Conv2D(256, (2, 2), strides = 1, activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(MaxPooling2D(pool_size=(2, 2), strides = 2, padding='same')) 

model.add(Conv2D(256, (2, 2), strides = 1, activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(MaxPooling2D(pool_size=(2, 2), strides = 2, padding='same'))

model.add(Flatten())

model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha=0.1))  

model.add(Dense(10, activation='linear'))
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(1))

In [ ]:
model.compile(loss = losses.mae,   
              optimizer = optimizers.Adam(learning_rate = 0.001),
              metrics = ['mae'])

In [ ]:
model.summary()

In [ ]:
epochs = 42

In [ ]:
model.fit(X_train, y_train, epochs=epochs)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
a, b = np.polyfit(y_test, y_pred, 1)

plt.scatter(y_test, y_pred)

plt.plot(y_test, a*y_test+b)
plt.xlabel('chronological age')
plt.ylabel('predicted age')

In [ ]:
from scipy.stats import pearsonr
corr, _ = pearsonr(y_test, y_pred)
print('Pearsons correlation: %.3f' % corr)

In [ ]:
from sklearn.metrics import r2_score 

In [ ]:
r2_score(y_test, y_pred)

In [ ]:
plt.figure(figsize = (6,6))
sns.boxplot(y = y_pred)
plt.title('predicted age')

In [ ]:
plt.figure(figsize = (6,6))
sns.boxplot(y = y_test)
plt.title('chronological age')